In [ ]:
%matplotlib inline
import pandas as pd
from rich import print
import numpy as np
import matplotlib.pyplot as plt
#from quickbin import bin2d
from scipy.stats import binned_statistic_2d
import pdr
from astropy.visualization import ZScaleInterval
from astropy import wcs as pywcs
import warnings


In [ ]:
def angularSeparation(ra1, dec1, ra2, dec2):
    d2r = np.pi/180.
    ra2deg = 1./d2r

    d1 = dec1*d2r
    d2 = dec2*d2r

    r1 = ra1*d2r
    r2 = ra2*d2r

    a = np.sin((d2-d1)/2.)**2.+np.cos(d1)*np.cos(d2)*np.sin((r2-r1)/2.)**2.
    r = 2*np.arcsin(np.sqrt(a))

    return r*ra2deg

In [ ]:
asp = pd.read_parquet('/Users/cm/github/gPhoton2/gPhoton/aspect/boresight_nosplit.parquet')
ra,dec = (155.794018, 61.612775)
sep = angularSeparation(ra,dec,
                        asp['ra0'].values,asp['dec0'].values)
ix = np.where(sep<0.5)
asp.iloc[ix]

In [ ]:
tbl1 = pd.read_parquet('data/e09147/e09147-nd-0-catalog.parquet')
tbl1

In [ ]:
img1 = pdr.read('data/e09147/e09147-nd-ffull-b00-image-r.fits')
plt.figure(figsize=(8,8))
plt.imshow(ZScaleInterval()(img1['CNT']),cmap="binary_r",origin="lower");
plt.plot(catpos[:,0],catpos[:,1],'yx')
plt.xlim([1500,1600])
plt.ylim([1500,1600])

In [ ]:
hdr = img1['CNT_HEADER']
wcs = pywcs.WCS(naxis=hdr['NAXIS'])
wcs.wcs.cdelt = [hdr['CDELT1'],
                 hdr['CDELT2']]
wcs.wcs.ctype = [hdr['CTYPE1'],
                 hdr['CTYPE2']]
wcs.wcs.crpix = [hdr['CRPIX1'],
                 hdr['CRPIX2']]
wcs.wcs.crval = [hdr['CRVAL1'],
                 hdr['CRVAL2']]

imgpos = wcs.wcs_world2pix([[ra,dec]],1) # set the origin to FITS standard
imgx,imgy = imgpos[0]
imsz = np.shape(img1['CNT'])[-2:]

# crop on the subframe
# noting that image coordinates and numpy coordinates are flipped
boxsz = 50
x1, x2, y1, y2 = (max(int(imgy - boxsz), 0),
                  min(int(imgy + boxsz), imsz[0]),
                  max(int(imgx - boxsz), 0),
                  min(int(imgx + boxsz), imsz[1]))
plt.plot([50],[50],'rx')
plt.title('TYC 4144-329-1')
plt.imshow(ZScaleInterval()(img1['CNT'][x1:x2, y1:y2]), origin="lower", cmap="Greys_r")

In [ ]:
imgpos = wcs.wcs_world2pix([[155.791120, 61.612065]],1) # set the origin to FITS standard
imgx,imgy = imgpos[0]
imsz = np.shape(img1['CNT'])[-2:]

# crop on the subframe
# noting that image coordinates and numpy coordinates are flipped
boxsz = 50
x1, x2, y1, y2 = (max(int(imgy - boxsz), 0),
                  min(int(imgy + boxsz), imsz[0]),
                  max(int(imgx - boxsz), 0),
                  min(int(imgx + boxsz), imsz[1]))
#plt.plot([50],[50],'rx')
plt.figure(figsize=(5,5))
plt.title('TYC 4144-329-2')
plt.imshow(ZScaleInterval()(img1['CNT']), origin="lower", cmap="Greys_r")
plt.xlim([y1,y2])
plt.ylim([x1,x2])

plt.plot(catpos[:,0][ix],catpos[:,1][ix],'yx')

In [ ]:
catpos = wcs.wcs_world2pix(list(zip(tbl1['RA'].values,tbl1['DEC'].values)),1) # set the origin to FITS standard
ix = np.where((np.isfinite(tbl1['NUV_MDL_SIGMA'])) & 
              (tbl1['NUV_MDL_SIGMA']<10) & (tbl1['NUV_MDL_SIGMA']>0.2) &
              (tbl1['NUV_MDL_CPS']>1e-03) &
              (tbl1['NUV_MDL_SIGMA']) & (tbl1['NUV_MDL_SIGMA']) &
              (tbl1['NUV_SOFTEDGE_FLAG_A6']==0) & (tbl1['NUV_HARDEDGE_FLAG_A6']==0) &
              (tbl1['NUV_GHOST_FLAG_A6']==0) & (tbl1['NUV_HOTSPOT_FLAG_A6']==0))

In [ ]:
# HMG: "I think that’s due to the choice of scale in the image! If you bin it right and chose a color scale 
# from max to mean, I was surprised how well it’s actually separated. Color in the image is NUV photons 
# binned up, white dots are single FUV photons."
plt.imshow(img1['CNT'], origin="lower", cmap="Greys_r", 
           vmin=np.mean(img1['CNT'][x1:x2, y1:y2]), vmax=np.max(img1['CNT'][x1:x2, y1:y2]))
plt.xlim([y1,y2])
plt.ylim([x1,x2])
plt.plot(catpos[:,0][ix],catpos[:,1][ix],'yx')

In [ ]:
cat_ix = np.where((catpos[:,0]>y1) & (catpos[:,0]<y2) &
                  (catpos[:,1]>x1) & (catpos[:,1]<x2))
plt.plot(catpos[cat_ix][:,0],catpos[cat_ix][:,1],'yx')

In [ ]:
def counts2mag(cps, band):
    scale = 18.82 if band == 'FUV' else 20.08
    with np.errstate(invalid='ignore'):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mag = -2.5 * np.log10(cps) + scale
    return mag

In [ ]:
counts2mag(tbl1.iloc[cat_ix]['NUV_MDL_CPS'].values,'NUV')

In [ ]:
tbl1.iloc[cat_ix][['NUV_MDL_CPS','NUV_MDL_SIGMA','NUV_MDL_BKG_CPS']]

In [ ]:
foo = tbl1.iloc[ix][['NUV_MDL_CPS','NUV_MDL_SIGMA','NUV_MDL_BKG_CPS']]
foo['NUV_MDL_MAG'] = counts2mag(tbl1.iloc[ix]['NUV_MDL_CPS'].values,'NUV')

In [ ]:
plt.hist(tbl1['NUV_MDL_SIGMA'].iloc[ix],range=[0,10],bins=100);
plt.xlabel('SIGMA (arcsec)')

In [ ]:
candidate_ix = np.where(
    (np.isfinite(tbl1['NUV_MDL_SIGMA'])) & 
    (tbl1['NUV_MDL_SIGMA']<10) & (tbl1['NUV_MDL_SIGMA']>5) &
    (tbl1['NUV_MDL_CPS']>1e-03) &
    (tbl1['NUV_MDL_SIGMA']) & (tbl1['NUV_MDL_SIGMA']) &
    (tbl1['NUV_SOFTEDGE_FLAG_A6']==0) & (tbl1['NUV_HARDEDGE_FLAG_A6']==0) &
    (tbl1['NUV_GHOST_FLAG_A6']==0) & (tbl1['NUV_HOTSPOT_FLAG_A6']==0))# &
    # (counts2mag(tbl1['NUV_MDL_CPS'].values,'NUV')<18))
len(candidate_ix[0])

In [ ]:
for cix in candidate_ix[0]:
    ra,dec = tbl1.iloc[cix][['RA','DEC']]
    imgpos = wcs.wcs_world2pix([[ra,dec]],1) # set the origin to FITS standard
    imgx,imgy = imgpos[0]
    imsz = np.shape(img1['CNT'])[-2:]
    
    # crop on the subframe
    # noting that image coordinates and numpy coordinates are flipped
    boxsz = 50
    x1, x2, y1, y2 = (max(int(imgy - boxsz), 0),
                      min(int(imgy + boxsz), imsz[0]),
                      max(int(imgx - boxsz), 0),
                      min(int(imgx + boxsz), imsz[1]))
    #plt.plot([50],[50],'rx')
    plt.figure(figsize=(5,5))
    plt.title(cix)
    
    plt.imshow(img1['CNT'], origin="lower", cmap="Greys_r", 
           vmin=np.mean(img1['CNT'][x1:x2, y1:y2]), vmax=np.max(img1['CNT'][x1:x2, y1:y2]))
    
    plt.xlim([y1,y2])
    plt.ylim([x1,x2])
    plt.plot(catpos[:,0][ix],catpos[:,1][ix],'yx')